In [ ]:
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import multiprocessing
import torch
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import denoise_wavelet
import random
import math
from PIL import Image
from collections import deque
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, label, generate_binary_structure
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.ndimage import label as ndi_label, binary_dilation

/home/lingfeng/Desktop/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_mean_pit_area(image):
    """
    Calcula el área media de fosas (pits) en la imagen, descartando aquellas que toquen
    el borde o estén adyacentes a fosas/colinas previas.
    """
    # Convierte la imagen a numpy y escala a grises uint8
    pixels = np.uint8(np.round(Perspectiver.rgb_to_grayscale(Perspectiver.normalize_to_uint8(image))))
    MAXROW, MAXCOL = pixels.shape

    pit_mask = np.zeros_like(pixels, dtype=bool)  # Máscara para fosas
    hill_mask = np.zeros_like(pixels, dtype=bool)  # Máscara para colinas
    pit_areas = []

    # Estructura 4-conexa
    structure = np.array([[0, 1, 0],
                          [1, 1, 1],
                          [0, 1, 0]], dtype=bool)

    # Procesa cada intensidad en orden ascendente
    for intensity in np.sort(np.unique(pixels)):
        mask = (pixels == intensity)  # Máscara binaria para la intensidad actual
        # Etiqueta componentes conexas usando ndi_label
        labeled, num_features = ndi_label(mask, structure=structure)

        for comp in range(1, num_features + 1):
            comp_mask = (labeled == comp)
            coords = np.argwhere(comp_mask)
            # Si la componente toca el borde, se clasifica como colina
            if (np.any(coords[:, 0] == 0) or np.any(coords[:, 0] == MAXROW - 1) or
                np.any(coords[:, 1] == 0) or np.any(coords[:, 1] == MAXCOL - 1)):
                hill_mask |= comp_mask
                continue

            # Obtiene vecinos 4-conexos mediante dilatación
            dilated = binary_dilation(comp_mask, structure=structure)
            border = dilated & ~comp_mask
            # Si es adyacente a fosas o colinas previas, clasifica como colina
            if np.any(pit_mask & border) or np.any(hill_mask & border):
                hill_mask |= comp_mask
            else:
                pit_mask |= comp_mask
                pit_areas.append(coords.shape[0])
    
    return np.mean(pit_areas) if pit_areas else -1

In [3]:
dataset = NEUDataset(set="train", transform=None, seed=1, scale=0.5)

# Probar con un DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Intentar iterar sobre el dataloader
for images, labels in dataloader:
    print("Iteración exitosa.")
    print("Imagen shape:", images.shape)
    print("Etiqueta shape:", labels.shape)
    break 

Iteración exitosa.
Imagen shape: torch.Size([4, 1, 100, 100])
Etiqueta shape: torch.Size([4, 6])


In [4]:
image, label = dataset.__getitem__(index=random.randint(2, len(dataset)))

'''
image = Image.open("metal_dataset/test/Pitted/PS_108.bmp")
original_width, original_height = image.size
scale = 0.5
image = image.resize((max(1, int(original_width * scale)), max(1, int(original_height * scale))))
original_image = Perspectiver.grayscale_to_rgb(Perspectiver.normalize_to_uint8(np.array(image)))
'''
original_image = Perspectiver.grayscale_to_rgb(Perspectiver.normalize_to_uint8(image.detach().cpu().numpy()[0]))

In [5]:
image = Image.open("metal_dataset/valid/Inclusion/In_118.bmp")
original_width, original_height = image.size
scale = 0.5
image = image.resize((max(1, int(original_width * scale)), max(1, int(original_height * scale))))
original_image = Perspectiver.grayscale_to_rgb(Perspectiver.normalize_to_uint8(np.array(image)))

In [6]:
def optimize_image(image, n_tries: int = 100):
    """
    Optimizes the mean shift parameters to maximize the metric: (Silhouette Score / Number of Clusters).

    Args:
        image (np.array): The input image with shape (200, 200, 3).
        n_tries (int): The number of optimization trials to run.

    Returns:
        dict: The best parameters and the best score.
    """

    # Define the objective function for Optuna
    def objective(trial):
        # Suggest parameters for spatial radius and color radius
        sp = trial.suggest_float("sp", 1.0, 60.0)  # Spatial window radius
        sr = trial.suggest_float("sr", 1.0, 60.0)  # Color window radius
        k = trial.suggest_int("k", 5, 30.0)
        # Apply mean shift filtering
        after = Perspectiver.kmeansClustering(Perspectiver.meanShift(image, sp, sr), k=k)

        return get_mean_pit_area(after)

    # Determine the number of CPU threads
    n_jobs = multiprocessing.cpu_count()-2
    print(f"Using {n_jobs} CPU threads for parallel optimization.")

    # Run the optimization with parallel trials
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_tries, n_jobs=n_jobs)

    # Return the best parameters and the best score
    return {
        "best_params": study.best_params,
        "best_score": study.best_value
    }

In [7]:
def plot_barchartImage(image):
    x = np.arange(image.shape[0])
    y = np.arange(image.shape[1])
    x, y = np.meshgrid(x, y)

    # Flatten arrays for plotting
    x = x.flatten()
    y = y.flatten()
    z = np.zeros_like(x)
    dx = dy = np.ones_like(x)
    dz = image.flatten()

    # Plot the 3D bar chart
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.bar3d(x, y, z, dx, dy, dz, shade=True)

    # Add labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Value')
    ax.set_title('3D Bar Chart of (200, 200) Array')

    plt.show()

In [8]:
BEST = optimize_image(original_image, n_tries=800)["best_params"]
BEST

[I 2025-02-05 18:22:44,595] A new study created in memory with name: no-name-46af292a-4a93-493d-be83-f4b000da7f4d


Using 6 CPU threads for parallel optimization.


[W 2025-02-05 18:22:45,114] Trial 3 failed with parameters: {'sp': 17.760863201932036, 'sr': 53.720073792168826, 'k': 21} because of the following error: NameError("name 'ndi_label' is not defined").
Traceback (most recent call last):
  File "/home/lingfeng/Desktop/pytorch/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_40370/2762780192.py", line 22, in objective
    return get_mean_pit_area(after)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_40370/529941716.py", line 23, in get_mean_pit_area
    labeled, num_features = ndi_label(mask, structure=structure)
                            ^^^^^^^^^
NameError: name 'ndi_label' is not defined
[W 2025-02-05 18:22:45,121] Trial 3 failed with value None.
[W 2025-02-05 18:22:45,132] Trial 1 failed with parameters: {'sp': 26.067163771525884, 'sr': 1.9211646172921188, 'k': 10} because of the following error: Name

NameError: name 'ndi_label' is not defined

In [ ]:
plot_barchartImage(Perspectiver.rgb_to_grayscale(original_image))

In [ ]:
clustered_image = Perspectiver.meanShift(original_image, BEST["sp"], BEST["sr"])
#clustered_image = Perspectiver.meanShift(original_image, 5, 6)
Perspectiver.plotComparison(original_image, clustered_image, titleBefore=label)

kmeans_image = Perspectiver.kmeansClustering(clustered_image, k = BEST["k"])
#kmeans_image = Perspectiver.kmeansClustering(clustered_image, k = 7)
Perspectiver.plotComparison(clustered_image, kmeans_image, titleBefore=label)

plot_barchartImage(Perspectiver.rgb_to_grayscale(kmeans_image))
Perspectiver.evaluate_clustering(original_image, kmeans_image)
get_mean_pit_area(kmeans_image)

In [11]:
def transformar_matriz(matriz):
    """
    Transforma una matriz con valores en [0,255] a una de [0,127].
    Si el valor <= 127 se mantiene, y si > 127 se mapea linealmente:
    127 -> 127 y 255 -> 0.
    """
    # Convertir a flotante para cálculos precisos
    out = matriz.astype(np.float32).copy()
    # Crear máscara para los valores mayores que 127
    mask = out > 127
    # Aplicar transformación lineal a los valores superiores a 127
    out[mask] = 127 - (out[mask] - 127) * (127 / 128)
    # Redondear y convertir a entero sin signo (0-255)
    return np.round(out).astype(np.uint8)

def distancia_minima_vecinos(arr):
    """
    Calcula para cada elemento la distancia mínima al vecino inmediato.
    En los extremos se usa el único vecino.
    """
    arr = np.array(arr)
    n = arr.size
    if n < 2:
        return np.zeros_like(arr)
    
    # Diferencias absolutas entre elementos adyacentes.
    dif = np.abs(np.diff(arr))
    
    # Inicializar el resultado.
    res = np.empty(n, dtype=arr.dtype)
    res[0] = dif[0]       # Primer elemento: sólo tiene el vecino derecho.
    res[-1] = dif[-1]     # Último elemento: sólo tiene el vecino izquierdo.
    
    # Para los elementos intermedios, se toma el mínimo de la diferencia con el vecino izquierdo y derecho.
    res[1:-1] = np.minimum(dif[:-1], dif[1:])
    
    return res

def distance_vector(clusters):
    ret = []
    for i in range(len(clusters)):
        if i == 0: ret.append(np.abs(clusters[i]-clusters[i+1]))
        if i == (len(clusters) - 1): ret.append(np.abs(clusters[i]-clusters[i-1]))
        else: ret.append(min(np.abs(clusters[i]-clusters[i-1]), np.abs(clusters[i]-clusters[i+1])))
    return ret

def plot_zonas_picos(matriz, umbral=None):
    """
    Detecta zonas de cambios bruscos (picos) en una matriz 2D mediante la magnitud del gradiente.
    Plotea la matriz en escala de grises y marca en azul los puntos donde la magnitud del gradiente
    supera el umbral definido.
    
    Parámetros:
      matriz: array 2D de entrada.
      umbral: valor umbral para considerar un cambio brusco. Si es None, se usa 80% del máximo.
    """
    # Calcular gradiente en las direcciones y (filas) y x (columnas)
    grad_y, grad_x = np.gradient(matriz.astype(float))
    # Magnitud del gradiente
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)
    
    # Definir umbral: 80% del máximo de la magnitud, si no se proporciona uno
    if umbral is None:
        umbral = 0.2 * np.max(grad_mag)
    
    # Índices donde la magnitud del gradiente supera el umbral
    indices = np.nonzero(grad_mag > umbral)
    
    # Plotear la imagen y superponer los puntos con cambios bruscos en azul
    plt.figure(figsize=(8, 6))
    plt.imshow(matriz, cmap='gray')
    plt.scatter(indices[1], indices[0], s=80, facecolors='none', edgecolors='blue', label='Zona de picos')
    plt.title("Zonas de picos (cambios bruscos) en azul")
    plt.legend()
    plt.show()

def variacion_media(arr):
    media = sum(arr) / len(arr)
    return sum(abs(x - media) for x in arr) / len(arr)

def plotear_imagen(array2d):
    """
    Plotea un array 2D como imagen en escala de grises.
    """
    plt.imshow(array2d, cmap='gray')
    plt.colorbar()  # Barra de color para referencia de intensidades
    plt.title("Imagen 2D")
    plt.show()

In [ ]:
import numpy as np
from collections import deque

def calcular_area_fosas(image):
    pixels = np.uint8(np.round(Perspectiver.rgb_to_grayscale(Perspectiver.normalize_to_uint8(image))))
    clusters = np.sort(np.unique(pixels))

    numero_de_fosas = 0
    area_de_fosas = 0

    registro_de_fosas = []

    directions = {"right": (1,0), "right": (-1,0), :(0,1), (0,-1)}

    MAXROW, MAXCOL = pixels.shape
    for cluster in clusters:

        print(f"Altura del cluster: {cluster}")
        
        rows, cols = np.where(pixels == cluster)
        positions = list(zip(rows, cols))

        print(positions)

        break

calcular_area_fosas(kmeans_image)

In [12]:
def anormal_ratio(image):
    pixels = np.uint8(np.round(Perspectiver.rgb_to_grayscale(Perspectiver.normalize_to_uint8(image))))
    print(f"Imagen entrante: \n {pixels}")
    #pixels = transformar_matriz(pixels)
    plotear_imagen(pixels)

    clusters = np.sort(np.unique(pixels))
    print(f"Clusters: {len(clusters)}, distribucion: \n {clusters} ")

    
    plot_zonas_picos(pixels)

In [ ]:
anormal_ratio(kmeans_image)